# Additional Exercise

Due: **12:00 18 July 2022**

**Online submission** at via [ILIAS](https://www.ilias.uni-koeln.de/ilias/goto_uk_exc_4593683.html) in the directory Exercises / Übungen -> Submission of Exercises / Rückgabe des Übungsblätter



### Marcel Günther 7348156, Nick Leonhardt 7348384

## Information

This additional exercise may be used for **one** of two reasons:

* make up for the attendance requirement (if you missed more than two tutorials), or
* replace lowest grade in exercises (mentioned to you via email)

For the latter, you may submit the solutions to this like any other exercise this year. For experience, it is advantageous to still use the following procedure.

If you need to make up for the attendance requirement, you must use a specific method of submission utilising [github](https://github.com/) and a makefile. If you are using Windows, please ensure you work either in [cygwin](https://www.cygwin.com/) or [Windows Subsystem for Linux](https://docs.microsoft.com/en-us/windows/wsl/install), since `make` does not work in Windows.

1. create a repository on github exclusively for this exercise
2. fully complete (or at least attempt) each question in a jupyter notebook
   * name it `[name]_additional_exercise_solution.ipynb`
3. restart and clear all notebook ouputs so it contains just the markdown and code cells (no plots or other outputs)
4. create a makefile to execute the notebook, compile to a pdf, and remove any temporary files
   * I should only need to run `$make` in order to build your submission
   * **do not** submit a compiled pdf of the solution
5. submit a file `[name].txt` containing the link to your git repository

Although make is a rather archaic tool, `make` is very useful to perform low-level compilation. It actually began as an easier method to compile C** and fortran code. Common compilation routines are identified using *suffix rules*. **For an additional 10%, write your own suffix rule to compile `.ipynb` files into `.pdf`.**

## `MakeFile`s

What a makefile does is simplify a workflow using terminal commands. As mentioned above, it is useful to compile documentation/code if changes where made in the source. One can in fact use this for newer formats such as IPython notebooks (`.ipynb` files).

A makefile typically goes in the top directory of your project. You can find an introduction to makefiles [here](https://makefiletutorial.com/). Basically how a makefile works is that you have a series of *targets* that execute a specific set of commands. For example, if your makefile contains,

```
info:
    @echo "This is a makefile."

command:
    @echo "This is a command."
```

> *Note:* Make will by default also print each command. Beginning the command with `@` suppresses this functionality.

you can make either target `info` or target `command`. Running `make info` in the terminal from the top directory will print `This is a makefile.` to the terminal. Likewise running `make command` will print `This is a command.` to the terminal. Not specifying a target will make the first target in the file (so `make info` is the same as `make` in this example). For this reason, it is customary to set the first target as the default workflow. If our makefile now contains,

```
all: info command

info:
    @echo "This is a makefile."

command:
    @echo "Some commands to execute:"
    echo "..."
    @echo "It has now completed."
```

running `make` from the terminal will print,

```
This is a makefile.
Some commands to execute:
echo "..."
...
It has now completed.
```

Each target can run multiple commands, but each line is run in a separate subshell. This means if you want to string together commands (for example if you need to change directories), you have to write them in one line as `command 1 ; command 2 ; ...`.

This should be most everything you need to write your own makefile. Please contact a TA if you run into any major issues.

> Ensure each command is indented with a **tab** character (not *space* characters). This is one of the finicky aspects of using a makefile...

> Note that linux operating systems use GNU make with `bash` or `dash` as a shell. Macintosh products typically use `zsh`.

## Working with a notebook from the terminal

Most operations you can perform on the notebook are available from the terminal. The command to work with notebooks is [nbconvert](https://nbconvert.readthedocs.io/en/latest/usage.html). This can convert between file types, depending on your needs. For example, executing a notebook from the terminal can be done using,

```
jupyter nbconvert --to notebook --execute notebook.ipynb
```

You can see a list of the available format types in the aforementioned link.

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from numpy import inf
from scipy import interpolate
from scipy import stats

## the *Hipparcos* catalogue

*Hipparcos* (HIgh Precision PARallax COllecting Satelite) was a very important stellar survey mission between 1989 and 1993. It was the first astrometric mission of its kind, and it's precise observations allowed for unprecedented astrometrical calculations within the Milky Way. The observations of faint stars have since been (*vastly*) improved upon by the current *Gaia* mission, though the brightest stars were still most accurately observed by *Hipparcos*.

The following examples will utilise a subset of a more recent reduction of the *Hipparcos* [data](https://vizier.u-strasbg.fr/viz-bin/VizieR-3?-source=I/311/hip2) ([van Leeuwen 2007](https://arxiv.org/pdf/0708.1752.pdf)) to emphasize how work with astronomical data within python, and hopefully teach a thing or two about stellar populations. The dataset we use are the stars with five-parameter astrometric solutions, meaning the solutions have been constrained for RA and DEC, proper motion in RA and DEC, and parallax. We will also highlight some of the features of various python packages so you can decide which method you prefer.

This dataset has 15 columns, with the titles as a comment in the first line. The columns (and units) are,

 - name in the *Hipparcos* catalog HIP
 - right ascentions RA ($\mathrm{RA}$; degrees)
 - error in right ascention sigma_RA ($\sigma_\mathrm{RA}$; milli-arcseconds)
 - declination DEC ($\mathrm{DEC}$; degrees)
 - error in declination sigma_DEC ($\sigma_\mathrm{DEC}$; milli-arcseconds)
 - parallax PLX ($\varpi$; in milli-arcseconds)
 - error in parallax sigma_PLX ($\sigma_\mathrm{\varpi}$; in milli-arcseconds)
 - right ascention proper motion PM_RA ($\mu_\mathrm{RA}$; milli-arcseconds per year)
 - error in right ascention proper motion sigma_PM_RA ($\sigma_{\mu, \mathrm{RA}}$; milli-arcseconds per year)
 - declination proper motion PM_DEC ($\mu_\mathrm{DEC}$; milli-arcseconds per year)
 - error in declination proper motion sigma_PM_DEC ($\sigma_{\mu, \mathrm{DEC}}$; milli-arcseconds per year)
 - *Hipparcos* magnitude Hp (dex)
   - *aka* V-band magnitude
 - error in magnitude sigma_Hp (dex)
 - B-V color (dex)
 - error in B-V color sigma_B-V (dex)
 - V-I color (dex)

### Calculating distance

A major benefit to precise astrometric observations is an accurate estimate of the parallax. This allows us to calculate the distance to each star and it reveal the structure of the Milky Way.

Begin by using the parallax ($\varpi$) to calculate the distance in parsecs (pc) to each star. It is important to remember that the parallax has units of milli-arcseconds (mas), so the distance to the star is calculated by:

$$ d = 1000 \Big(\frac{\mathrm{mas}}{\varpi}\Big) \ \mathrm{pc} $$

Use this formula to find some interesting trends in the *Hipparcos* catalog. Are there any issues with the dataset? **5 points**

> *Note:* Each degree contains 60 arcminutes, which each contain 60 arcseconds, so there are 3 600 000 milli-arcseconds in 1 degree.

> *Note:* The Milky Way has a diameter of $\sim 40~000$ pc and Earth is located at a radius of $\sim 8~000$ pc.

Can you find what is the most common distance to a star? What is the standard deviation of this value? What can you determine from these values? **5 points**

In [ ]:
data = pd.read_csv('hipparcos2.csv', sep=',')
data

In [ ]:
filtering_PLX = data['PLX']>data['sigma_PLX']
filtered_PLX = filtering_PLX*data['PLX']

In [ ]:
filtered_PLX.replace([np.inf, 0, -np.inf], np.nan, inplace=True)

### issues with the data set is that there are some inf values and for a few values the error is bigger than the value itself. Therefore we are filtering the values where the error is bigger then the value it self and replace +-inf and 0 with nan. So these values will be ignored.

In [ ]:
data['Distance (pc)'] = (1000/filtered_PLX)

In [ ]:
distance = np.asarray(data['Distance (pc)'])

In [ ]:
plt.hist(distance, bins=1000)
plt.title('Distance of the stars in the Hipparcos catalog in pc ')
plt.xlabel('Distance in pc')
plt.ylabel('Quantity')
plt.tight_layout()
plt.axvline(x=100,color='red',linestyle='--')
plt.grid()

In [ ]:
print('the most common distance is around 100 pc')
print('the mean distance is',data['Distance (pc)'].mean(skipna=True),'pc')
print('the std of the distance is',data['Distance (pc)'].std(skipna=True),'pc')

### It can be can sayd that the most common distance is about 100pc, which also can be seen in the histogram above (red line). But there are huge differents between the distances which can be seen at the standard deviation value and also in the plot above. 

### Utilising error

Error is a major aspect of observational data, since there are so many sources (instrumentational and physical) to consider. Following the procedure of van Leeuwen (2007), let us examine the error in parallax and its trends in our dataset. Relative error $\big(\frac{\sigma_q}{q}\big)$ is an important metric when our data spans multiple orders of magnitude. What is the distribution of relative error in parallax (you may want to make y log-spaced)? What does this mean in terms of *Hipparcos*' precision? **10 points**

In [ ]:
data['PLX'].replace([np.inf, 0, -np.inf], np.nan, inplace=True)
data['sigma_PLX'].replace([np.inf, 0, -np.inf], np.nan, inplace=True)

In [ ]:
data['Relativ error PLX'] = abs(data['sigma_PLX']/data['PLX'])
relativ_error= np.asarray(data['Relativ error PLX'])

In [ ]:
plt.hist(relativ_error,bins=np.arange(0,5,0.01))
plt.title('Distribution of Relative Error of Parallax ')
plt.xlabel('Relative Error')
plt.yscale('log')
plt.ylabel('Quantity in log scale')
plt.tight_layout()
plt.xlim((0,5))
plt.show()

### By looking at plot above which is in y log scaled, it can be seen that there is still an log distribution. This means he distribution of relative error in parallax seems to be a double log distribution. In terms of Hipparcos' precision it can be said, that it is not good because too much values have a too high relative error.

The error in parallax is typically related to the brightness of the star (from the statistics of photon counts). A view of this trend can be seen below (keeping the relative parallax error below 5%; similar to Figure 11 in van Leewen 2007). Can you find a fit to this relation, including error? **10 points**

![relative error](hipparcos_plx_err.png)

In [ ]:
rel_error_filtering = data['Relativ error PLX'] <= 0.05 
rel_error_filtered = rel_error_filtering* data['sigma_PLX']

In [ ]:
rel_error_filtered_np = np.asarray(rel_error_filtered)
magnitude = np.asarray(data['Hp'])

In [ ]:
x=magnitude
y=rel_error_filtered_np
plt.rcParams["figure.figsize"] = (8,8)
plt.scatter(x,y,s=0.1)
plt.yscale('log')
plt.ylim((0.1,8))
plt.xlim((1,13))
plt.xlabel("HP")
plt.ylabel("sigma_PLX")
plt.gca().invert_xaxis()

In [ ]:
x=magnitude
y=rel_error_filtered_np
plt.rcParams["figure.figsize"] = (8,8)
plt.plot(np.unique(x), np.poly1d(np.polyfit(-x, y, 1))(np.unique(x)),color='red')
plt.scatter(x,y,s=0.1)
#plt.yscale('log')
plt.ylim((0.1,8))
plt.xlim((1,13))
plt.xlabel("HP")
plt.ylabel("sigma_PLX")
plt.gca().invert_xaxis()

### we could only find a fit for the plot without ylog scaled. 

In [ ]:
print('the fitted line has a and b of:',np.polyfit(-x, y, 1))

In [ ]:
print('The residual value returned is the sum of the squares of the fit errors:', np.sum((np.polyval(np.polyfit(-x, y, 1), -x) - y)**2))

### The above value tells us, that this fit is not good and there is a big deviation in the values.

Can you plot a histogram of the relative errors in $\mathrm{RA}, \ \mathrm{DEC},$ and $\mathrm{PLX}$ in a single figure? **10 points**

In [ ]:
data['Relativer error RA'] = abs(data['sigma_RA']/data['RA'])
RA = np.asarray(data['Relativer error RA'])

data['Relativer error DEC'] = abs(data['sigma_DEC']/data['DEC'])
DEC = np.asarray(data['Relativer error DEC'])


data['Relativer error PLX'] = abs(data['sigma_PLX']/data['PLX'])
data['Relativer error PLX'].replace([np.inf, 0, -np.inf], np.nan, inplace=True)
PLX = np.asarray(data['Relativer error PLX'])



In [ ]:
plt.figure()
plt.hist(RA,bins=np.arange(0,0.5,0.01),label='RA',alpha=0.5)
plt.hist(DEC,bins=np.arange(0,0.5,0.01),label='DEC',alpha=0.5)
plt.hist(PLX,bins=np.arange(0,0.5,0.01),label='PLX',alpha=0.5)
plt.legend()
plt.xlabel('relative errors')
plt.ylabel('Quantity')
plt.title('histogram of the relative errors in $\mathrm{RA}, \ \mathrm{DEC},$ and $\mathrm{PLX}$')
plt.xlim((0,0.5))
plt.show()

### By looking at the above histogram of the relative errors in $\mathrm{RA}, \ \mathrm{DEC},$ and $\mathrm{PLX}$, it can be said, that $\mathrm{RA}$ has the best precission and $\mathrm{PLX}$ the badest.

### Hertzsprung-Russell diagrams

#### Observational

Before calculating any intrinsic stellar properties from the observations, it is typically useful to plot a diagram of what you *observe*. (Observed here means the most basic properties that can be determined from the data. If you want to be pedantic, then you might suggest that the values in this table are not what is actually observed, but a reduction of the observed photon counts.) For this purpose we should plot a Hertzsprung-Russel diagram (HRD) to reveal the different stellar types. It is important not only to remember that a HRD has colour (B-V) on the x-axis and magnitude (Hp) on the y-axis, but also that the y-axis is reversed (the perks of using an antiquated unit). **5 points**

In [ ]:
data.plot.scatter(x = 'B-V',y = 'Hp',s=0.1)
plt.title('HRD of Hippacros Catalogue')
plt.gca().invert_yaxis()

### It can be seen a lot of Values where B-V is null, and also a lot of points overlapping. It is like a blurred HRD.

What do you see?

If you use the magnitude given in `hipparcos2.csv`, you might have a little difficulty determining the different regions of an HR diagram. One reason for this is that it is an apparent magnitude, so these results do not take into account reddening or extinction. Another issue, however, is that we don't filter the results that have a lot of error. Try plotting it again while limiting the relative parallax error to <20% and the color error to 5 mag. **5 points**

In [ ]:
PLX_error = data['sigma_PLX']/data['PLX']<0.2
Color_error = data['sigma_B-V']<0.5

PLX_Color_limit = PLX_error * Color_error

B_V_limit = data['B-V']*PLX_Color_limit
B_V_limit_np = np.asarray(B_V_limit)
Hp_limit = data['Hp']*PLX_Color_limit
Hp_limit_np = np.asarray(Hp_limit)
V_I_limit = data['V-I']*PLX_Color_limit
V_I_limit_np = np.asarray(V_I_limit)

In [ ]:
plt.scatter(x = B_V_limit_np, y = Hp_limit_np, s = 0.1)
plt.title('HRD of Hippacros Catalogue corrected')
plt.xlabel('B-V')
plt.ylabel('Hp')
plt.gca().invert_yaxis()

One trait now looks more prominent, and that is the suspiciously high number of stars with $ B-V = 0 $. This is called an artefact, and it is a result of some bad data or sloppy reduction. Since the author of this reduction is quite renowned in the community, the issue most likely lies with bad data for these stars. For the rest of the exercise, you may filter-out star with $B-V = 0$.

Now try plotting the HRD in terms of the intrinsic absolute magnitude, $ M_{Hp} $ (you should get something similar in shape to [this](https://www.cosmos.esa.int/documents/532822/573165/f3_5_005.pdf/90951100-6586-4ba7-a042-784f5845a279)). **Try also with the other colour $V-I$**. Remember that the absolute magnitude can be calculated by the formula:

$$
M_\mathrm{Hp} = m_\mathrm{Hp} - 5 \ \mathrm{log}_\mathrm{10}\Big(\frac{d}{pc}\Big) + 5
$$

This plot has a bit of an issue where there are many points plotted ontop of each other. Instead, see if you can plot the number density in a 2D histogram of of the following ways **10 points**

* bin the data in steps of 0.05 mag in $M_{Hp}$ and 0.01 mag in $B-V$, with the number density in the color bar. You might find the function `scipy.stats.binned_statistic_2d()` useful.
* Utilise the `hexbin` routine with an adequate choice of binning

In [ ]:
B_V_limit_np_new= np.where(B_V_limit_np==0, np.nan, B_V_limit_np)
V_I_limit_np_new= np.where(V_I_limit_np==0, np.nan, V_I_limit_np)

In [ ]:
data['MHP'] = (data['Hp'])-(5*np.log(data['Distance (pc)']))+5
MHP = np.asarray(data['MHP'])

In [ ]:
plt.scatter(x = B_V_limit_np_new, y = MHP, s = 0.1)
plt.title('HRD of Hippacros Catalogue corrected with B-V')
plt.xlabel('B-V')
plt.ylabel('absolute Magnitude')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
plt.scatter(x = V_I_limit_np_new, y = MHP, s = 0.1)
plt.title('HRD of Hippacros Catalogue corrected with V-I')
plt.xlabel('I-V')
plt.ylabel('absolute Magnitude')
plt.gca().invert_yaxis()

In [ ]:
B_V_new= np.where(B_V_limit_np_new==np.nan, 0, B_V_limit_np_new)
MHP_new= np.where(MHP==np.nan, 0, MHP)
z=np.arange(len(MHP_new))

#stats.binned_statistic_2d(B_V_new, MHP_new, values = z, statistic ='count', bins = [0.01, 0.05])
#binned statistic 2d ist not wokring properly, but we don't now whu

Very broadly we say that the features observed here are the main sequence (diagonal feature) and red clump (roughly spherical feature). Fit a line to the main sequence. Approximate the percentage of stars in the main sequence compared to the percentage in the red clump. **10 points**

> *Note:* This will not add up to 100%.

In [ ]:
plt.scatter(x = B_V_limit_np_new, y = MHP, s = 0.1)
plt.title('HRD of Hippacros Catalogue corrected with B-V')
plt.xlabel('B-V')
plt.ylabel('absolute Magnitude')
plt.plot(np.unique(x), np.poly1d(np.polyfit(x, -y, 1))(np.unique(x)),color='red')
plt.gca().invert_yaxis()
plt.show()

#### Theoretical

For theorists it is more illuminating to plot diagrams of intrinsic properties. This makes it easier to relate the observations to population synthesis or evolutionary models. Getting to these intrinsic stellar properties from our observations is typically quite involved, requiring some (not so trivial) estimate of the dust extinction and attenuation along the line of sight. We will assume there is no dust extinction for a very basic example.

A star's colour can be used to calculate its effective (surface) temperature. Disregarding any reddenning, we can estimate it using the formula ([Ballesteros 2012](https://arxiv.org/pdf/1201.1809.pdf)):

$$
T_\mathrm{eff} = 4600 \ \Big( \frac{1}{0.92(B-V)+0.62} + \frac{1}{0.92(B-V)+1.7} \Big) \ \mathrm{K}
$$

Luminosity is the intrinsic analog of magnitude. To put it in solar units, we need to use the absolute bolometric magnitude of the Sun ($M_\odot=4.74$) with the observed star's bolometric magnitude and extinction. Since our absolute magnitudes are in the V band, we will have to add a bolometric correction ($BC_\mathrm{Hp}$) to convert to the bolometric magnitude. Thus we have too many unknowns for our to make an accurate approximation. Current missions (ie. *Gaia*) get around this by utilising extremely-randomised trees, a regression method beyond the scope of this simple introduction. We will **not** use regression, but instead use an extremely-simplified approach where we neglect extinction and interpolate the bolometric correction (from `BC_hipparcos.csv`; [Masana et al. 2008](https://arxiv.org/pdf/astro-ph/0601049.pdf)) to get the formula:

$$
-2.5 \ \mathrm{log_{10}} \Big( \frac{L}{L_\odot} \Big) = M_\mathrm{Hp} + BC_\mathrm{Hp}(T_\mathrm{eff}) - M_\odot
$$

Finally, the stellar radii can be calculated using the standard blackbody relation:

$$
L = \Big( \frac{R}{R_\odot} \Big)^2 \Big( \frac{T_\mathrm{eff}}{T_{\mathrm{eff},\odot}} \Big)^4 \ L_\odot
$$

It might also help you to know the solar units: $L_\odot=3.828*10^{26} \ \mathrm{W}$, $R_\odot=6.956*10^8 \ \mathrm{m}$, $T_\odot=5780 \ \mathrm{K}$.

Perform a spline fit of the bolometric correction data. Correctly interpolate the BC and utilise the equations to create a 2D histogram (or hexbin plot) of $L$ as a function of $T_\mathrm{eff}$. **10 points**

> *hint:* Use units of $L_\odot$ and K.

> *hint:* Technically this fit is only valid for $T_\mathrm{eff} \in [3300, 8000]$ K.

In [ ]:
data_2 = pd.read_csv('BC_hipparcos.csv', sep=',')
BC = np.asarray(data_2['BC'])
T = np.asarray(data_2['# Teff'])

In [ ]:
f = interpolate.interp1d(T,BC)
T_new = np.arange(4000,8000,4000/len(MHP))
BC_new = f(T_new)
data['BC_new']=BC_new
data['T_new']=T_new

In [ ]:
MHP_new= np.where(MHP==np.nan, 0, MHP)
data['MHP_new']=MHP_new

In [ ]:
data['Luminosity'] = 10**((0.4*(4.74-data['MHP_new']-data['BC_new'])))
data['Luminosity'].replace([np.inf, np.nan, -np.inf], 0, inplace=True)

In [ ]:
L = np.asarray(data['Luminosity'])
T = np.asarray(data['T_new'])

In [ ]:
plt.hist2d(T,L)
#plt.plot(np.unique(T), np.poly1d(np.polyfit(T, L, 1))(np.unique(T)),color='red')
plt.xlabel('T(K)')
plt.ylabel('L')
plt.show()

Now try to perform a linear fit to the main sequence, accounting for error. Estimate the error in the fit. **10 points**

In [ ]:
plt.plot(np.unique(T), np.poly1d(np.polyfit(T, L, 1))(np.unique(T)),color='red')
plt.xlabel('T(K)')
plt.ylabel('L')

### We interpolated the BC values and calculated the Lumonisity. For the hist2d plot we could not find a good solution. The Main Error here is a lot of nan in the data. But the linear fit without the hist2d plot gave a good result as expected. The Luminosity arises if the temperature does.

Figure out how to isolate the stars on the main sequence. What is the probability for a star to be on the main sequence? Use the bootstrapping technique to determine the constrain the error in this approximation. **10 points**

### maybe with the proper motion

In [ ]:
filter_PM_RA = data['PM_RA']>10
filter_PM_DEC = data['PM_DEC']>10
filter_PM= filter_PM_RA*filter_PM_DEC
B_V_last = B_V_limit_np_new*filter_PM
MHP_last = MHP*filter_PM

In [ ]:
plt.scatter(x = B_V_last, y = MHP_last, s = 0.1)
plt.title('HRD of Hippacros Catalogue corrected with B-V')
plt.xlabel('B-V')
plt.ylabel('absolute Magnitude')
plt.gca().invert_yaxis()
plt.show()

### could not isolate the main sequence with the proper motion